In [24]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

def fatigue_log_cp_model(time, cp, w_prime, k, alpha, F):
    return cp + (w_prime / ((time + k) ** alpha)) - F * np.log(time + 1)

def estimate_ftp(data, cp20=None, power_prediction=None):
    """
    Estimates FTP (or Critical Power) from either a list/series of wattages or a dictionary of time-wattage pairs.
    Optionally predicts power output for a given duration.
    
    Args:
        data (list, np.ndarray, pd.Series, or dict):
            - If list/array/series: it is assumed to be power data over time, and mean max power is computed.
            - If dict: should be {duration (seconds): mean max power}.
        cp20 (float, optional): 20-minute CP estimate (if available), used for a better initial guess.
        power_prediction (int, optional): Duration in seconds to predict power output.
        
    Returns:
        dict: Estimated CP, model parameters, and optional power prediction.
    """
    
    if isinstance(data, (list, np.ndarray, pd.Series)):
        # Compute mean max power for standard durations if a time series of power is provided
        time_windows = [60, 300, 600, 1200, 1800, 3600, 7200]  # Common durations (1m, 5m, 10m, etc.)
        df = pd.DataFrame({'watts': data})
        df['time'] = np.arange(len(data))  # Assume sequential time indices
        
        power_data = {tw: peak_avg_wattage(df, tw) for tw in time_windows if len(data) >= tw}
    elif isinstance(data, dict):
        # Use provided mean max power values directly
        power_data = data
    else:
        raise ValueError("Unsupported data format. Provide a list/series of wattages or a dictionary of time-power values.")
    
    short_durations_seconds = np.array(list(power_data.keys()))
    short_power_array = np.array(list(power_data.values()))
    
    # Default to 95% of 20-min power if cp20 is not provided
    if cp20 is None:
        cp20 = short_power_array[short_durations_seconds == 1200][0] * 0.95 if 1200 in short_durations_seconds else np.median(short_power_array) * 0.95
    
    # Set initial guesses
    initial_cp = 0.95 * cp20
    initial_w_prime = (short_power_array[0] - initial_cp) * short_durations_seconds[0]
    initial_k = 300  # Mid-range offset
    initial_alpha = 1.0
    initial_F = 10.0
    
    # Set conservative bounds
    bounds = ([cp20 * 0.85, 2000, 10, 0.5, 1.0],
              [cp20 * .98, 20000, 3000, 2.0, 20.0])
    
    # Ensure initial conditions are within bounds
    p0 = np.clip([initial_cp, initial_w_prime, initial_k, initial_alpha, initial_F], bounds[0], bounds[1])
    
    popt, _ = curve_fit(fatigue_log_cp_model, short_durations_seconds, short_power_array, p0=p0, bounds=bounds, maxfev=10000)
    
    cp, w_prime, k, alpha, F = popt
    
    result = {
        "CP": cp,
        "W'": w_prime,
        "k": k,
        "alpha": alpha,
        "F": F
    }
    if power_prediction is not None:
        result["Predicted Power"] = fatigue_log_cp_model(power_prediction, cp, w_prime, k, alpha, F)
        result["Predicted Power Duration"] = power_prediction
    return {k: float(v) for k, v in result.items()}

def peak_avg_wattage(df, time_window):
    df['watts'] = df['watts'].fillna(0)
    df['rolling_avg'] = df['watts'].rolling(window=time_window, min_periods=1).mean()
    return df['rolling_avg'].max()

# Example Usage:
power_dict = {60: 435, 300: 388, 600: 332, 1200: 328, 1800: 270, 3600: 244, 7200: 232}
estimated_ftp = estimate_ftp(power_dict, power_prediction=5000)
print(f"Estimated FTP: {estimated_ftp['CP']:.1f}")
print(f"Estimated Power at {estimated_ftp['Predicted Power Duration']}: {estimated_ftp['Predicted Power']:.1f}")




Estimated FTP: 264.9
Estimated Power at 5000.0: 238.5


In [26]:
df = pd.read_csv('/Users/joshuagordon/Documents/sandbox/strava-im-estimator/ironman-estimator/example_data/example_outdoor_bike.csv')
estimated_ftp = estimate_ftp(df['watts'])
print(f"Estimated FTP: {estimated_ftp['CP']:.1f}")

Estimated FTP: 207.3
